In [2]:
import pandas as pd
from gensim.models import Word2Vec
import re
from nltk.stem import SnowballStemmer, WordNetLemmatizer
stemmer = SnowballStemmer('english')
import nltk
nltk.download('wordnet')
import numpy as np
lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gigimancinelli./nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
criminality = ['theft', 'deviance', 'indictment', 'criminalization', 'guilt', 'government', 'tort', 
               'criminalness', 'criminalism', 'imprisonment', 'punishment', 'law', 'banditry', 'lawlessness', 
               'statutory', 'illegality', 'murder', 'immorality', 'sectarianism', 'dishonesty', 
               'impunity', 'criminalisation', 'brutality', 'thuggery', 'trial', 'recklessness', 'militancy', 
               'decriminalization', 'sentence', 'bigotry', 'extremism', 'viciousness', 'duplicity', 
               'xenophobia', 'mysonygy', 'stupidity', 'hypocrisy', 'irresponsibility', 'racism',
               'illegalities', 'coercion', 'vigilantism', 'imprisonment', 'plaintiff', 'criminology', 
               'felony', 'desertion', 'charge', 'power', 'wrong']
mental = ['schizophrenia', 'depression', 'bipolar', 'disorder', 'antisocial', 'personality', 'psychosis', 
          'psychotherapy', 'dementia', 'psychopathy', 'disability', 'anxiety', 'illness', 'alcoholism', 'brain', 
          'diagnosis', 'mind', 'symptoms', 'delusions', 'hallucinations', 'chronic', 'attention', 'deficit', 
          'hyperactivity', 'psychiatrists', 'disease', 'stress', 'epilepsy', 'agoraphobia', 'suffering', 
          'dysthymia', 'hypomania', 'sclerosis', 'psychiatric', 'psychological', 'disabilities', 'paraphilia', 
          'ocd', 'psychotic', 'syndrome', 'delusional', 'community', 'eating', 'disorder', 'anorexia', 'bulimia', 
          'nervosa', 'sickness', 'substance', 'abuse']

In [4]:
len(criminality)

50

In [5]:
len(mental)

50

In [7]:
model = Word2Vec.load("./model/word2vec_before_floyd.model")

In [8]:
df = pd.read_csv("./data/mpv_clean_pattern_stop.csv")

In [9]:
df.head()

,Unnamed: 0,name,gender,race,age,date,after_Floyd,city,state,cause_of_death,...,wapo_armed,wapo_threat_level,wapo_flee,wapo_body_camera,initial_reason,text,authors,TextColumn,cleaned,sentences
0,0,Clyde E. Young,Male,White,28.0,2024-02-22,True,West Alton,MO,Gunshot,...,NaN,Used Weapon,Not Fleeing,NaN,Murder (suspect),['A Godfrey man who had been charged with murd...,"[['Mike Koziatek'], []]",a godfrey man who had been charged with murd...,godfrey man who had been charged with murder i...,"[['godfrey', 'man', 'charg', 'murder', 'shoot'..."
1,1,Clyde E. Young,Male,White,28.0,2024-02-22,True,West Alton,MO,Gunshot,...,NaN,Used Weapon,Not Fleeing,NaN,Murder (suspect),"['The man, identified as Clyde Young, was char...","[['Mike Koziatek'], []]",the man identified as clyde young was char...,the man identified as clyde young was charged ...,"[['man', 'identifi', 'clyde', 'young', 'charg'..."
2,2,Jamarcus Brown,Male,Black,35.0,2024-02-21,True,Lake Cormorant,MS,Gunshot,...,NaN,Attack,Car,Victim Facebook Live Video,Traffic Stop,"['DESOTO COUNTY, Miss. (WMC) - Mississippi Bur...","[[], ['Lakiya Scott'], []]",desoto county miss wmc mississippi bur...,desoto county miss wmc mississippi bureau of i...,"[['desoto', 'counti', 'miss', 'wmc', 'mississi..."
3,3,Jamarcus Brown,Male,Black,35.0,2024-02-21,True,Lake Cormorant,MS,Gunshot,...,NaN,Attack,Car,Victim Facebook Live Video,Traffic Stop,['Close Get email notifications on {{subject}}...,"[[], ['Lakiya Scott'], []]",close get email notifications on subject ...,close get on subject are only sent and only if...,"[['close', 'get', 'subject', 'sent', 'new']]"
4,4,Jamarcus Brown,Male,Black,35.0,2024-02-21,True,Lake Cormorant,MS,Gunshot,...,NaN,Attack,Car,Victim Facebook Live Video,Traffic Stop,['The Carlos Moore Law Group said it has been ...,"[[], ['Lakiya Scott'], []]",the carlos moore law group said it has been ...,the carlos moore law group said it has been re...,"[['carlo', 'moor', 'law', 'group', 'said', 're..."


In [10]:
df = df[df.after_Floyd == 0]
df.head()

,Unnamed: 0,name,gender,race,age,date,after_Floyd,city,state,cause_of_death,...,wapo_armed,wapo_threat_level,wapo_flee,wapo_body_camera,initial_reason,text,authors,TextColumn,cleaned,sentences
9540,9540,Dion Johnson,Male,Black,28.0,2020-05-25,False,Phoenix,AZ,Gunshot,...,No Object,Other,Not Fleeing,No,Asleep in car (blocking traffic),['A 28-year-old man died after being shot by a...,[[]],a XXXXXX year old man died after being shot ...,year old man died after being shot by an arizo...,"[['year', 'old', 'man', 'die', 'shot', 'arizon..."
9541,9541,Reymar Gagarin,Male,Native Hawaiian and Pacific Islander,35.0,2020-05-25,False,Modesto,CA,Gunshot,...,Toy Weapon,Attack,Not Fleeing,Yes,Suspicious Person,['One man was killed after an officer-involved...,"[['Kcra Staff', 'Https', 'Www.Facebook.Com']]",one man was killed after an officer involved...,one man was killed after an officer involved s...,"[['one', 'man', 'kill', 'offic', 'involv', 'sh..."
9542,9542,George Floyd,Male,Black,46.0,2020-05-25,False,Minneapolis,MN,Physical Restraint,...,No Object,Other,Not Fleeing,Yes,Forged check,['The FBI and authorities in Minnesota have la...,"[['Kenya Evelyn'], []]",the fbi and authorities in minnesota have la...,the fbi and authorities in minnesota have laun...,"[['fbi', 'author', 'minnesota', 'launch', 'inv..."
9543,9543,George Floyd,Male,Black,46.0,2020-05-25,False,Minneapolis,MN,Physical Restraint,...,No Object,Other,Not Fleeing,Yes,Forged check,[''],"[['Kenya Evelyn'], []]",,NaN,[]
9544,9544,Anthony Grove,Male,Unknown race,52.0,2020-05-24,False,Kila,MT,Gunshot,...,Gun,Attack,Not Fleeing,No,Disturbance,"[""MISSOULA, Mont. (AP) — Montana authorities s...","[['Associated Press', 'May', 'At P.M.']]",missoula mont ap montana authorities s...,missoula mont ap montana authorities say man w...,"[['missoula', 'mont', 'ap', 'montana', 'author..."


In [12]:
def create_embedding(text):
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    text = text.strip()
    embeddings = [model.wv[word] for word in text.split()]
    if len(embeddings) == 0:
        return 0
    emb = sum(embeddings) / len(embeddings)
    return emb

In [13]:
df['emb'] = df['sentences'].apply(create_embedding)

In [14]:
df.columns

Index(['Unnamed: 0', 'name', 'gender', 'race', 'age', 'date', 'after_Floyd',
       'city', 'state', 'cause_of_death', 'circumstances', 'news_urls',
       'signs_of_mental_illness', 'officer_charged', 'allegedly_armed',
       'wapo_armed', 'wapo_threat_level', 'wapo_flee', 'wapo_body_camera',
       'initial_reason', 'text', 'authors', 'TextColumn', 'cleaned',
       'sentences', 'emb'],
      dtype='object')

In [15]:
df.allegedly_armed.unique()

array(['Unarmed/Did Not Have Actual Weapon', 'Allegedly Armed', 'Unclear',
       'Vehicle'], dtype=object)

In [16]:
df.iloc[0].emb

array([-0.3198648 ,  0.08247469,  0.18260302, -0.2787408 , -0.0804928 ,
        0.05568931, -0.4530957 ,  0.29003087, -0.53236145, -0.36274666,
       -0.40975833,  0.3240186 , -0.0286827 ,  0.88560385,  0.32822067,
       -0.17601909,  0.2727202 , -0.5614346 , -0.24288826,  0.06777334,
       -0.34952256,  0.41065133, -0.37001294,  0.73117584,  0.9430155 ,
       -0.34203944, -0.28407827,  0.49517295,  0.0787483 , -0.10519397,
       -0.73428106,  0.24089716, -0.67473745,  0.15421695,  0.47148043,
       -0.2889184 ,  0.34201157, -0.6465494 ,  0.1518729 ,  0.36646792,
        0.41242552, -0.37393782,  0.9084713 , -0.21653017, -0.0474827 ,
        0.3630811 ,  0.49496642,  0.9139698 ,  0.27827856, -0.14859009,
       -0.03470332, -0.54532486, -0.21267942, -0.23289508,  0.47497144,
       -0.0194388 , -0.6311741 ,  0.29847297, -0.17055185,  0.5632564 ,
        0.07129121,  0.03380478, -0.07430896,  0.00979098,  0.11119524,
       -0.2086293 ,  0.1146762 , -0.47177115, -0.26992035, -0.23

In [17]:
def create_embedding_dimension_lem(l):
    lemmatized = [lemmatizer.lemmatize(w) for w in l] 
    embeddings = [model.wv[word] for word in lemmatized if word in model.wv]
    if len(embeddings) == 0:
        return 0
    emb = sum(embeddings) / len(embeddings)
    return emb

In [18]:
def create_embedding_dimension(l):
    stemmed = [stemmer.stem(w) for w in l] 
    embeddings = [model.wv[word] for word in stemmed if word in model.wv]
    if len(embeddings) == 0:
        return 0
    emb = sum(embeddings) / len(embeddings)
    return emb

In [19]:
criminal_dimension = create_embedding_dimension(criminality)
mental_dimension = create_embedding_dimension(mental)

In [20]:
criminal_dimension_lem = create_embedding_dimension_lem(criminality)
mental_dimension_lem = create_embedding_dimension_lem(mental)

In [21]:
criminal_dimension

array([ 0.16450872, -0.5754865 , -0.23153691, -0.01478629, -0.35141757,
        0.60716   ,  0.03755226, -0.02729095,  0.06187922,  0.20751795,
        0.06461097,  0.65000856,  0.02833593,  0.43889856, -0.06187674,
       -0.10422292,  0.2400769 ,  0.1459391 , -0.31555203, -0.26375467,
        0.38144684,  0.02127731, -0.3139545 ,  0.7098815 ,  0.21419723,
       -0.1659213 , -0.27394807, -0.2232126 , -0.45663685,  0.04503582,
        0.20316668, -0.2708475 ,  0.08834095, -0.04616502, -0.10659347,
        0.21436906,  0.24270985, -0.27659386,  0.11043117,  0.16319868,
        0.28631097, -0.18019697,  0.5817863 , -0.01376015, -0.42161602,
        0.07092178,  0.2667014 , -0.03237163, -0.03101639, -0.10037074,
       -0.09763314, -0.54776347,  0.01986785,  0.21024032, -0.01625049,
        0.05152525, -0.1717684 ,  0.10497241, -0.18291683,  0.03406792,
       -0.20309687,  0.19475761, -0.26603618, -0.05790906, -0.33787602,
       -0.0632514 , -0.1514766 , -0.08661533, -0.36230874, -0.15

In [22]:
mental_dimension

array([-0.02536458, -0.13179745, -0.23405544,  0.10342605, -0.26183134,
        0.4039547 ,  0.39049435,  0.25825715, -0.01933487, -0.08694631,
        0.093891  ,  0.37841862,  0.06689981,  0.09644082,  0.07104122,
       -0.15498051,  0.3004254 , -0.04578625,  0.02012766, -0.2388821 ,
        0.5850873 , -0.02011888,  0.07660916,  0.37409458, -0.24882817,
        0.30912018, -0.37777427, -0.06455736, -0.22267419, -0.14666554,
        0.05574656, -0.17977662, -0.08084849, -0.2684792 ,  0.10461606,
        0.24892792, -0.08421971, -0.20174411,  0.08974564,  0.23545484,
        0.23178698, -0.2748743 ,  0.3894324 , -0.09778746, -0.17312132,
        0.31738997,  0.17797728,  0.05830799,  0.07919475,  0.08225547,
       -0.31325617, -0.46771994, -0.10921598, -0.01445774,  0.1482184 ,
       -0.09530978, -0.13221039, -0.06766609, -0.16415673,  0.35737067,
       -0.26081952, -0.01121674, -0.1699271 , -0.03266311, -0.21063502,
        0.09975211,  0.06435907,  0.00562243, -0.4401948 ,  0.15

In [23]:
def cosine_similarity(v1, v2):
    dot_ = np.dot(v1, v2)
    norm1 = np.linalg.norm(v1)
    norm2 = np.linalg.norm(v2)
    if norm1 > 0 and norm2 > 0:
        return dot_ / (norm1 * norm2)
    else:
        return 0.0

In [24]:
cosine_similarity(criminal_dimension, mental_dimension)

0.6429258

In [25]:
cosine_similarity(criminal_dimension_lem, mental_dimension_lem)

0.56635743

In [26]:
df['cosine_criminal'] = df['emb'].apply(lambda x: cosine_similarity(criminal_dimension, x))
df['cosine_mental'] = df['emb'].apply(lambda x: cosine_similarity(mental_dimension, x))

In [27]:
df['cosine_criminal_lem'] = df['emb'].apply(lambda x: cosine_similarity(criminal_dimension_lem, x))
df['cosine_mental_lem'] = df['emb'].apply(lambda x: cosine_similarity(mental_dimension_lem, x))

In [28]:
average_criminal_white = df[df.race == 'White'].cosine_criminal.sum() / len(df[df.race == 'White'])
average_criminal_black = df[df.race == 'Black'].cosine_criminal.sum() / len(df[df.race == 'Black'])
average_mental_white = df[df.race == 'White'].cosine_mental.sum() / len(df[df.race == 'White'])
average_mental_black = df[df.race == 'Black'].cosine_mental.sum() / len(df[df.race == 'Black'])

print(average_criminal_white)
print(average_criminal_black)
print(average_mental_white)
print(average_mental_black)

0.4734303734742141
0.49602597587332
0.3774017695080411
0.4005139732615944


In [29]:
average_criminal_white_lem = df[df.race == 'White'].cosine_criminal_lem.sum() / len(df[df.race == 'White'])
average_criminal_black_lem = df[df.race == 'Black'].cosine_criminal_lem.sum() / len(df[df.race == 'Black'])
average_mental_white_lem = df[df.race == 'White'].cosine_mental_lem.sum() / len(df[df.race == 'White'])
average_mental_black_lem = df[df.race == 'Black'].cosine_mental_lem.sum() / len(df[df.race == 'Black'])

print(average_criminal_white_lem)
print(average_criminal_black_lem)
print(average_mental_white_lem)
print(average_mental_black_lem)

0.4501289630586799
0.4760091093456628
0.3761035698057895
0.40377352453505394


In [30]:
def extract_sign_mental(col):
    if col == 'Yes' or col == 'Drug or Alcohol Use' or col == 'History of Drug or Alcohol Use':
        return 1
    else:
        return 0
    
def extract_sign_armed(col):
    if col == 'Allegedly Armed' or col == 'Vehicle':
        return 1
    else:
        return 0

In [31]:
df['mental'] = df['signs_of_mental_illness'].apply(extract_sign_mental)
df['criminal'] = df['allegedly_armed'].apply(extract_sign_armed)

In [32]:
proxy_criminal_white = df[df.race == 'White'].criminal.sum() / len(df[df.race == 'White'])
proxy_criminal_black = df[df.race == 'Black'].criminal.sum() / len(df[df.race == 'Black'])
proxy_mental_white = df[df.race == 'White'].mental.sum() / len(df[df.race == 'White'])
proxy_mental_black = df[df.race == 'Black'].mental.sum() / len(df[df.race == 'Black'])

In [33]:
print(proxy_criminal_white)
print(proxy_criminal_black)
print(proxy_mental_white)
print(proxy_mental_black)

0.757223113964687
0.6761133603238867
0.31942215088282505
0.16310005783689993


In [34]:
print(f"Proportion of white victims armed: {proxy_criminal_white}. Criminal bias in articles for white victims: {average_criminal_white}")
print(f"Proportion of black victims armed: {proxy_criminal_black}. Criminal bias in articles for black victims: {average_criminal_black}")
print(f"Proportion of white victims with signs of mental illness: {proxy_mental_white}. Mental bias in articles for white victims: {average_mental_white}")
print(f"Proportion of black victims with signs of mental illness: {proxy_mental_black}. Mental bias in articles for black victims: {average_mental_black}")

Proportion of white victims armed: 0.757223113964687. Criminal bias in articles for white victims: 0.4734303734742141
Proportion of black victims armed: 0.6761133603238867. Criminal bias in articles for black victims: 0.49602597587332
Proportion of white victims with signs of mental illness: 0.31942215088282505. Mental bias in articles for white victims: 0.3774017695080411
Proportion of black victims with signs of mental illness: 0.16310005783689993. Mental bias in articles for black victims: 0.4005139732615944
